In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [8]:
from google.colab import files
uploaded = files.upload()

Saving creditcard.csv to creditcard (2).csv


In [9]:
credit_card_data = pd.read_csv('creditcard (2).csv')
credit_card_data.head

<bound method NDFrame.head of             Time         V1         V2  ...       V28  Amount  Class
0            0.0  -1.359807  -0.072781  ... -0.021053  149.62      0
1            0.0   1.191857   0.266151  ...  0.014724    2.69      0
2            1.0  -1.358354  -1.340163  ... -0.059752  378.66      0
3            1.0  -0.966272  -0.185226  ...  0.061458  123.50      0
4            2.0  -1.158233   0.877737  ...  0.215153   69.99      0
...          ...        ...        ...  ...       ...     ...    ...
284802  172786.0 -11.881118  10.071785  ...  0.823731    0.77      0
284803  172787.0  -0.732789  -0.055080  ... -0.053527   24.79      0
284804  172788.0   1.919565  -0.301254  ... -0.026561   67.88      0
284805  172788.0  -0.240440   0.530483  ...  0.104533   10.00      0
284806  172792.0  -0.533413  -0.189733  ...  0.013649  217.00      0

[284807 rows x 31 columns]>

In [14]:
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [15]:
data = credit_card_data.iloc[:,:-1]
targets = credit_card_data.iloc[:,-1]

print(data.shape)
print(targets.shape)

(284807, 30)
(284807,)


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=.3)

In [18]:
train_data, validation_data, train_targets, validation_targets = train_test_split(train_data, train_targets, test_size=.2)

In [19]:
mean = np.mean(train_data)
std = np.std(train_data)

train_data -= mean
train_data /= std

validation_data -= mean
validation_data /= std

test_data -= mean
test_data /= std

In [21]:
from keras import models, layers

In [22]:
model = models.Sequential()
model.add(layers.Dense(10, input_shape=(train_data.shape[1],), activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [23]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
history = model.fit(train_data, train_targets, epochs=100, validation_data=(validation_data, validation_targets))

Epoch 1/100
4985/4985 [==============================] - 8s 1ms/step - loss: 0.0478 - accuracy: 0.9987 - val_loss: 0.0049 - val_accuracy: 0.9993
Epoch 2/100
4985/4985 [==============================] - 7s 1ms/step - loss: 0.0078 - accuracy: 0.9992 - val_loss: 0.0056 - val_accuracy: 0.9994
Epoch 3/100
4985/4985 [==============================] - 7s 1ms/step - loss: 0.0065 - accuracy: 0.9994 - val_loss: 0.0065 - val_accuracy: 0.9994
Epoch 4/100
4985/4985 [==============================] - 7s 1ms/step - loss: 0.0059 - accuracy: 0.9994 - val_loss: 0.0052 - val_accuracy: 0.9994
Epoch 5/100
4985/4985 [==============================] - 7s 1ms/step - loss: 0.0063 - accuracy: 0.9993 - val_loss: 0.0040 - val_accuracy: 0.9994
Epoch 6/100
4985/4985 [==============================] - 7s 1ms/step - loss: 0.0065 - accuracy: 0.9994 - val_loss: 0.0052 - val_accuracy: 0.9994
Epoch 7/100
4985/4985 [==============================] - 7s 1ms/step - loss: 0.0052 - accuracy: 0.9995 - val_loss: 0.0043 - val_ac

In [39]:
model.evaluate(test_data, test_targets)

2671/2671 [==============================] - 2s 918us/step - loss: 0.0076 - accuracy: 0.9993


[0.007646390702575445, 0.9993329048156738]

In [40]:
prediction = model.predict(test_data)
prediction.shape

(85443, 1)

In [41]:
prediction = prediction.reshape(85443)

In [43]:
output_df = pd.DataFrame({'Actual': test_targets, 'Prediction': np.round(prediction)})
output_df.head()

,Actual,Prediction
90032,0,0.0
263193,0,0.0
203267,0,0.0
257501,0,0.0
62091,0,0.0


In [45]:
output_df.shape

(85443, 2)

In [47]:
filter = (output_df.loc[:, 'Actual'] == output_df.loc[:, 'Prediction'])

output_df[filter].shape

(85386, 2)

In [49]:
print(f'Accuracy is {(output_df[filter].shape[0]/output_df.shape[0])*100}%')

Accuracy is 99.93328885923948%


In [50]:
train_targets.value_counts()

0    159218
1       273
Name: Class, dtype: int64